In [1]:
import pandas as pd
import re
import string

In [97]:
scraped= pd.read_csv('Hotel Location Table Mapping.csv',encoding='latin1')
hotels=pd.read_csv('Hotel Location.csv',encoding='latin1')
scraped_ngrams=pd.read_csv('submission.csv',encoding='latin1')

old=pd.read_csv('old.csv',encoding='latin1')
pivot=pd.read_csv('pivot.csv',encoding='latin1')

In [4]:
def clean_text(text):
    """
    This function uses regular expressions to 
        - remove links characters
        - remove html characters,
        - remove escape sequences,
        - remove capitalization,
        - any extra white space from each text and then converts them to lowercase.

    Input:
    text: original text
          datatype: string

    Output:
    texts: modified text
           datatype: string
    """
    # replace links with url-web
    pattern_url = 'http[s]?://(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9A-Fa-f][0-9A-Fa-f]))+'
    subs_url = 'url-web'
    text = re.sub(pattern_url, subs_url, text)
    # replace the html characters with " "
    text = re.sub('<.*?>#()&', ' ', text)
    text = re.sub('&', ' ', text)
#     text = re.sub('(', ' ', text)
#     text = re.sub(')', ' ', text)
    
    text=re.sub('Nan', ' ', text)
    # replace escape sequence with space
    escape_seq = ["\r", "\n",'\a', '\b', '\f', "\`", '\”', '\t']
    for i in escape_seq:
        text = re.sub(i, " ", text)
    # Remove Capitalization
    text = text.lower()
    # plit and join the words
    text=' '.join(text.split())
    
    return text

In [6]:
def clean(text):
    
    # removing paragraph numbers
    text = re.sub('[0-9]+.\t','',str(text))
    # removing new line characters
    text = re.sub('\n ','',str(text))
    text = re.sub('\n',' ',str(text))
    # removing apostrophes
    text = re.sub("'s",'',str(text))
    # removing hyphens
    text = re.sub("#",' ',str(text))
    text = re.sub("@",'',str(text))
    
    # removing quotation marks
    text = re.sub('\"','',str(text))
    # removing salutations
    text = re.sub("Mr\.",'Mr',str(text))
    text = re.sub("Mrs\.",'Mrs',str(text))
    # removing any reference to outside text
    text = re.sub("[\(\[].*?[\)\]]#", "", str(text))
    
    return text

In [7]:
def remove_punctuation(post):
    return ''.join([l for l in post if l not in string.punctuation])

In [8]:
scraped['description'] = scraped['description'].astype(str).apply(clean_text)
scraped['description'] = scraped['description'].astype(str).apply(remove_punctuation)

In [98]:
def domain_name(url):
    return url.split("www.")[-1].split("//")[-1].split(".")[0]

In [12]:
selected_rows = scraped[scraped['hotel_name'].isnull()]

In [31]:
selected_rows .head(3)

,scraped_url,agency_slug,country,title,description,inclusions,hotel_name,hotel_mapped,domain
0,https://www.flightcentre.co.za/product/14197346,flightcentre,maldives,Adaaran Select Meedhupparu,your maldives holiday package includes return flights from johannesburg approximate taxes return domestic flight speedboat transfers 7 nights accommodation at the 4star adaaran select meedhupparu in a beachfront villa all meals selected alcoholic and nonalcoholic drinks daily exclusions items of a personal nature items not mentioned gratuities and porterage meals drinks not mentioned tourssightseeing not mentioned car hire with the promise of an incomparable holiday in the tropics adaaran select meedhupparu one of the best maldives luxury resorts offers blissful accommodation for the discerning traveller to the maldives bask under the healing sunshine and soak up the alluring sights and sounds of a truly enchanting island in the tropics adaaran select meedhupparu offers tranquil settings brimming with luxury settle into a holiday full of breathtaking experiences that will leave you with memories to last a lifetime,\nYour Maldives Holiday Package includes:\n\nReturn flights from Johannesburg\nApproximate taxes\nReturn domestic flight & speedboat transfers\n7 nights' accommodation at the 4-star Adaaran Select Meedhupparu in a Beachfront Villa\nAll meals & selected alcoholic and non-alcoholic drinks daily\n\n,NaN,NaN,flightcentre
1,https://www.flightcentre.co.za/product/5415249,flightcentre,south africa,Family Fun on the Wild Coast,your eastern cape holiday package includes return flights from johannesburg to durban approximate taxes 3 days car hire kia picanto or similar 3 nights midweek accommodation on the wild coast at the 4star wild coast sun in a garden facing luxury room breakfast daily bonus complimentary access to wild waves water park please note 2 children eat breakfast and stay free when sharing with adults weekend upgrade from r660 per adult exclusions transfers items of a personal nature items not mentioned gratuities and porterage meals and drinks not mentioned tourssightseeing not mentioned less than 2 hours from durban you cant beat the location of the wild coast sun hotel situated in the coastal hub of port edward less than a twohour scenic drive along the south coast from durbans king shaka international airport this familyfriendly accommodation is just the place if youre looking for that relaxing room with a view with all the modern facilities one could ever need moms and dads can soak up the sun watching the kids play at the wild waves water park while couples can slip away to the tranquil oasis of the mangwanani african day spa,\nYour Eastern Cape Holiday Package includes:\n\nReturn flights from Johannesburg to Durban\nApproximate taxes\n3 days car hire (Kia Picanto or similar)\n3 nights' midweek accommodation on the Wild Coast at the 4-star Wild Coast Sun in a Garden Facing Luxury Room\nBreakfast daily\nBONUS: Complimentary access to Wild Waves Water Park\nPLEASE NOTE: 2 Children eat breakfast and stay free when sharing with adults\nWEEKEND UPGRADE: From R660 per adult\n\n,NaN,NaN,flightcentre
2,https://www.flightcentre.co.za/product/16385011,flightcentre,mauritius,Azuri Residences by Life in Blue,your mauritius holiday package includes return direct flights from durban approximate taxes return transfers 5 nights accommodation at the 4star azuri residences by life in blue in a deluxe apartment on a room only basis please note price is based on 4 people travelling together exclusions items of a personal nature items not mentioned gratuities and porterage meals and drinks not mentioned tourssightseeing not mentioned car hire this coastal village on the northeastern coast of the island in roches noires lined by an infinite turquoise sea and the blue sky will tinge your holidays with picturesque shades of blue opt for our highend seafront accommodation and enjoy your intimacy within an elegant setting kindle your senses an

In [32]:
nonselected_rows = scraped[scraped['hotel_name'].notbnull()]

In [33]:
nonselected_rows .head()

,scraped_url,agency_slug,country,title,description,inclusions,hotel_name,hotel_mapped
613,https://www.flightcentre.co.za/product/4054754,flightcentre,south africa,The Swakopmund Experience,your swakopmund holiday package includes return flights from johannesburg to windhoek approximate taxes 4 days group b car hire with unlimited mileage and max waivers 3 nights accommodation at the 3star beach hotel swakopmund breakfast daily bonus includes a seal and dolphin cruise and a half day living desert tour exclusions items of a personal nature items not mentioned gratuities porterage meals drinks not mentioned tourssightseeing not mentioned get the full swakopmund experience exploring the life both in the desolate sands and the cool atlantic ocean this historic and quaint seaside town will awe you with its charm location and excellent food,\nYour Swakopmund Holiday Package includes:\n\nReturn flights from Johannesburg to Windhoek\nApproximate taxes\n4 days Group B car hire with unlimited mileage and max waivers\n3 nights' accommodation at the 3-star Beach Hotel Swakopmund\nBreakfast daily\nBONUS: Includes a Seal and Dolphin Cruise and a Half Day Living Desert Tour\n\n,3-star beach hotel,NaN
614,https://www.flightcentre.co.za/product/15497971,flightcentre,mauritius,Friday Attitude,your mauritius holiday package includes return flights from johannesburg approximate taxes return airporthotel transfers 5 nights accommodation at the 3star friday attitude in a standard room breakfast lunch dinner selected drinks daily add 2 nights from r2350 per person exclusions items of a personal nature items not mentioned gratuities and porterage meals and drinks not mentioned tourssightseeing not mentioned car hire this 3star superior hotel is nestled in trou deau douce a quiet village along the east coast of mauritius,"\nYour Mauritius Holiday Package includes:\n\nReturn flights from Johannesburg\nApproximate taxes\nReturn airport/hotel transfers\n5 nights' accommodation at the 3-star Friday Attitude in a Standard Room\nBreakfast, lunch, dinner & selected drinks daily\nADD: 2 nights from R2350 per person\n\n","3-star friday attitude in a standard room\nbreakfast, lunch, dinner & selected drinks daily\nadd: 2 nights from r2350 per person\n\nhotel",NaN
615,https://www.flightcentre.co.za/product/5487199,flightcentre,united arab emirates,Golden Sands Hotel Apartments,your dubai holiday package includes return flights from cape town approximate taxes return private transfers 5 nights selfcatering accommodation at the 3star golden sands hotel apartments in a studio room on a room only basis free wifi optional addon a 24 hour hop on hop off dubai bus tour from r1020 per person exclusions items of a personal nature items not mentioned gratuities porterage meals drinks not mentioned tourssightseeing not mentioned car hire golden sands 3 hotel apartments offers a comprehensive range of apartment types to suit individuals as well as families coming to dubai for leisure or business or both golden sands a budget hotel apartment with its perfectly placed location offers ideal accommodation in dubai with all facilities and amenities,\nYour Dubai Holiday Package includes:\n\nReturn flights from Cape Town\nApproximate taxes\nReturn private transfers\n5 nights' self-catering accommodation at the 3-star Golden Sands Hotel Apartments in a Studio Room on a room only basis\nFree Wi-Fi\nOPTIONAL ADD-ON: A 24 Hour Hop On Hop Off Dubai Bus Tour from R1020 per person\n\n,3-star golden sands hotel,NaN
616,https://www.flightcentre.co.za/product/5487197,flightcentre,united arab emirates,Golden Sands Hotel Apartments,your dubai holiday package includes return flights from johannesburg approximate taxes return private transfers 5 nights selfcatering accommodation at the 3star golden sands hotel apartments in a studio room on a room only basis free wifi optional addon a 24 hour hop on hop off dubai bus tour from r1150 per person exclusions items of a personal nature items not ment

In [13]:
selected_rows['domain'] =selected_rows['scraped_url'].astype(str).apply(domain_name)

C:\Users\user\AppData\Local\Temp\ipykernel_9640\3696863612.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_rows['domain'] =selected_rows['scraped_url'].astype(str).apply(domain_name)


In [15]:
type_labels = list(selected_rows.domain.unique())

In [16]:
type_labels 

['flightcentre',
 'igotravel',
 'casterbridge-hollow',
 'elephant-plains-lodge',
 'idube-game-reserve',
 'marataba-trails-lodge',
 'hans-merensky-hotel-and-golf-estate',
 'bayethe-lodge',
 '4-day-fly-in-safari',
 'madikwe-hills-private-game-lodge',
 'phinda-homestead',
 'long-lee-manor',
 'camp-shawu',
 'thula-thula-private-game-reserve',
 'simbavati-river-lodge',
 'inn-on-the-square',
 'madikwe-safari-lodge',
 'country-boutique-hotel',
 'rhino-ridge-safari-lodge',
 'entabeni-lakeside-lodge',
 'kirkmans-kamp',
 'exeter-river-lodge',
 'ekuthuleni-lodge',
 'nedile-lodge',
 'leopard-hills-private-game-reserve',
 'thonga-beach-lodge',
 'nottens-bush-camp',
 'isibindi-zulu-lodge',
 'hillsnek-safari-camp',
 'camp-shonga',
 'leadwood-lodge',
 'nungubane-game-lodge',
 'impodimo-game-lodge',
 'alpine-heath-resort',
 'entabeni-kingfisher-lodge',
 'mandela-rhodes-place',
 'marataba-safari-lodge',
 'greenway-woods-resort-2980',
 'legend-golf-and-safari-resort',
 '2-day-fly-in-safari',
 'umkumbe-sa

In [17]:
pd.set_option('display.max_colwidth', -1)   
neew = selected_rows.loc[selected_rows['domain']=='igotravel']

C:\Users\user\AppData\Local\Temp\ipykernel_9640\3316049702.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [20]:
neew['hotels_new'] = neew['title']

C:\Users\user\AppData\Local\Temp\ipykernel_9640\1640489448.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neew['hotels_new'] = neew['title']


In [96]:
neew

,scraped_url,agency_slug,country,title,description,inclusions,hotel_name,hotel_mapped,domain,hotels_new
102,https://www.igotravel.co.za/holidays/13-day-egypt-tour/,igotravel,egypt,13 Day Egypt Tour,div classwoocommerceproductdetailsshortdescription p styletextalign centervisit the incredible pyramids of giza relax on a felucca as you cruise down the nile and discover the ancient wonders of aswan and luxor before spending a few days by the beautiful red sea in hurghada this is egypt at its best in 13 wonderful days choose to travel by felucca or nile river cruiser on this tour you have the option of a 2 night stay onboard a traditional felucca sail boat or 3 nights onboard a 5 star nile cruiser the details of both are noted in our itinerary if choosing to travel by 5 star nile cruiser please select the cruise upgrade prices amp dates for costsp p styletextalign centertrip highlightsbr hurghada  relax at the red sea swim and opt to snorkel or scuba dive in the clear coastal watersbr luxor west bank  discover exquisite tombs in the valley of the kings the towering colossi of memnon and the beautiful temple of queen hatshepsutbr aswan  take leisurely boat trip to agilika island to explore the romantic philae temple and wander around the colourful souqsbr kom ombo  visit the nile side temple of kom ombobr cairo  marvel at the legendary pyramids of giza explore the necropolis of saqqara and see king tuts treasures at the egyptian museumbr luxor  roam around the colossal temple of karnak and take an optional tour of the beautifully illuminated luxor temple at nightbr nile felucca sailing  sail the river nile on board a traditional felucca and spend two nights sleeping under a blanket of stars or upgrade to a 5 star nile cruisep div,"<div class=""blck col-sm-6"">\n<h3 class=""blck-ttl"">Inclusions</h3><p class=""blck-txt pb-0""></p><ul><br>\n<li>Breakfast daily</li><br>\n<li>2 Lunches</li><br>\n<li>2 Dinners</li><br>\n<li>Escorted by an Egyptian tour guide degree qualified in Egyptology</li><br>\n<li>4 Days' Hurghada - Red Sea stay - swim, sun and sand</li><br>\n<li>Airport arrival and departure transfer, day 1 and day 13</li><br>\n<li>Escorted by an Egyptian tour guide degree qualified in Egyptology</li><br>\n<li>9 Nights' in 4 - 5 star hotels</li><br>\n<li>2 Nights' aboard felucca (open deck)</li><br>\n<li>1 overnight train: Cairo - Aswan (Private 2 berth sleeper cabin)</li><br>\n<li>Touring of Cairo, Giza, Saqqara, Aswan, Luxor and Hughada</li><br>\n<li>All relevant transfer and transportation in private modern air-conditioned vehicles</li><br>\n<li>Set departure dates to choose from (Friday departures)</li><br>\n</ul> </div>",NaN,NaN,igotravel,13 Day Egypt Tour
103,https://www.igotravel.co.za/holidays/hilton-seychelles-northolme-resort/,igotravel,seychelles,Hilton Seychelles Northolme Resort,div classwoocommerceproductdetailsshortdescription p styletextalign centerauthor ian fleming wrote for your eyes only in this secluded 5 star hideaway small intimate charming as well as beautifully equipped private romantic villas await you villas raised on stilts providing uninterrupted views of the ocean white sandy beaches tropical gardens as well as crystal clear waters surround the hilton seychelles northolme resort amp spa this place is a truly stunning escape located on mahes north west coastline this is an adult only resort only kids over 13 years old are allowed to stay here hilton seychelles northolme resort amp spa features great attention to detail in the stunning wooden interiors featuring contemporary decor as well as original artwork special finishing touches include bathrobe slippers mineral water as well as fruit the new grand ocean pool villas are truly spectacular with an infinity private pool set into the sunwashed decking of each villa the striking spa is perched on granite boulders overlooking the gorgeous oceanp p styletextalign centerdiscover national monuments beautiful creole houses artists studios natio

In [23]:
pd.set_option('display.max_colwidth', -1)   
neew1 = selected_rows.loc[selected_rows['domain']=='packages']

C:\Users\user\AppData\Local\Temp\ipykernel_9640\2166239504.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [25]:
neew1['hotels_new'] = neew1['title']

C:\Users\user\AppData\Local\Temp\ipykernel_9640\2116465824.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neew1['hotels_new'] = neew1['title']


In [99]:
neew1['hotels_new']

KeyError: 'hotels_new'

In [27]:
frames = [neew, neew1]
result = pd.concat(frames) 

In [95]:
result['hotels_new']

102     13 Day Egypt Tour                               
103     Hilton Seychelles Northolme Resort              
104     Vienna, Budapest & Prague Insight Vacations Tour
105     Disneys Art of Animation                       
106     Safari Plains, Waterberg                        
                  ...                                   
6717    Pearl Beach Zanzibar                            
6733    My Blue Hotel                                   
6740    The Kingdom at Victoria Falls                   
6770    Adaaran Select Meedhupparu                      
6816    Veranda Pointe Aux Biches                       
Name: hotels_new, Length: 787, dtype: object

In [35]:
pd.set_option('display.max_colwidth', -1)   
neew2 = selected_rows.loc[selected_rows['domain']=='computravel']

C:\Users\user\AppData\Local\Temp\ipykernel_9640\3710132810.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [91]:
#neew2.head(5)

In [44]:
def findAfter (txt):
#len(txt.split(''))
    myList = txt.split(" ")
    for x in myList:
        if x=='4*':
            start = txt.find (x) #Sets start to the start of keyword
            end = start + len (x) #Sets end to the end of the keyword
            return txt [end:end+30] 
        elif x=='3*':
            start = txt.find (x) #Sets start to the start of keyword
            end = start + len (x) #Sets end to the end of the keyword
            return txt [end:end+30] 
        elif x=='2*':
            start = txt.find (x) #Sets start to the start of keyword
            end = start + len (x) #Sets end to the end of the keyword
            return txt [end:end+30] 
        elif x=='5*':
            start = txt.find (x) #Sets start to the start of keyword
            end = start + len (x) #Sets end to the end of the keyword
            return txt [end:end+30] 
        elif x=='5star':
            start = txt.find (x) #Sets start to the start of keyword
            end = start + len (x) #Sets end to the end of the keyword
            return txt [end:end+30]
    


In [100]:
neew2['value'] = neew2['title'] .astype(str).apply(findAfter)

C:\Users\user\AppData\Local\Temp\ipykernel_9640\204853310.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neew2['value'] = neew2['title'] .astype(str).apply(findAfter)


In [101]:
 neew2['hotels_new'] = neew2['value'].str.extract('(.+?)-')

C:\Users\user\AppData\Local\Temp\ipykernel_9640\1613221675.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neew2['hotels_new'] = neew2['value'].str.extract('(.+?)-')


In [72]:
#neew2['hotels_new'].head(40)

In [74]:
pd.set_option('display.max_colwidth', -1)   
neew3 = selected_rows.loc[selected_rows['domain']=='pentravel']

C:\Users\user\AppData\Local\Temp\ipykernel_9640\3074155294.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [77]:
neew3['hotels_new'] = neew3['title']

C:\Users\user\AppData\Local\Temp\ipykernel_9640\2782757151.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neew3['hotels_new'] = neew3['title']


In [78]:
def findAfter2 (txt):
#len(txt.split(''))
    myList = txt.split(" ")
    for x in myList:
        if x=='4-star':
            start = txt.find (x) #Sets start to the start of keyword
            end = start + len (x) #Sets end to the end of the keyword
            return txt [end:end+30] 
        elif x=='3star':
            start = txt.find (x) #Sets start to the start of keyword
            end = start + len (x) #Sets end to the end of the keyword
            return txt [end:end+30] 
        elif x=='4star':
            start = txt.find (x) #Sets start to the start of keyword
            end = start + len (x) #Sets end to the end of the keyword
            return txt [end:end+30] 
        elif x=='45star':
            start = txt.find (x) #Sets start to the start of keyword
            end = start + len (x) #Sets end to the end of the keyword
            return txt [end:end+30] 
        elif x=='5star':
            start = txt.find (x) #Sets start to the start of keyword
            end = start + len (x) #Sets end to the end of the keyword
            return txt [end:end+30]

In [79]:
neew4 = selected_rows.loc[selected_rows['domain']=='flightcentre']

In [80]:
neew4['hotels_new'] = neew4['description'] .astype(str).apply(findAfter2)

C:\Users\user\AppData\Local\Temp\ipykernel_9640\1046999580.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neew4['hotels_new'] = neew4['description'] .astype(str).apply(findAfter2)


In [86]:
neew4['hotels_new']=neew4['hotels_new'] .astype(str).apply(first3)

C:\Users\user\AppData\Local\Temp\ipykernel_9640\31037954.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neew4['hotels_new']=neew4['hotels_new'] .astype(str).apply(first3)


In [92]:
#neew4['hotels_new'].sample(40)

In [85]:
def first3(txx):
    lis=txx.split()[:3]
    new=' '.join(lis)
    return new

In [93]:
neew5 = selected_rows.loc[selected_rows['domain']=='leolapa']

In [94]:
neew5 

,scraped_url,agency_slug,country,title,description,inclusions,hotel_name,hotel_mapped,domain
572,https://www.africastay.com//leolapa,africastay,NaN,LeoLapa Amenities,span classdestinationpricefrom r550000 per nightspan20150101 20151231strongtraveller ratingstrong span classgradingspan classfa fastarspan span classfa fastarspan span classfa fastarspan span classfa fastarspan span classfa fastarspan spanstrongcurrent reviewsstrong 2p classrsubjectstrongtitlestrong the view from both houses is outstandingp classrsubjectstrongtitlestrong the house is enormous itp color808080leolapa is located in mpumalanga kruger national park we provide accommodation hotel information holiday specials packages photos rates and leolapa reviews book leolapa online and savepay securely with,NaN,NaN,NaN,leolapa


In [ ]:
frames = [result, neew1]
result = pd.concat(frames) 